In [2]:
import numpy as np 
import pandas as pd 
from sklearn.svm import SVC
import Cleanup


In [4]:
train_df = pd.read_csv("train_df.csv")
test_df = pd.read_csv("test_df.csv")


In [5]:
train_df = train_df.drop(['Unnamed: 0'],axis = 1)
test_df = test_df.drop(['Unnamed: 0'],axis = 1)

In [6]:
len(train_df['L-F_Pair'].unique())


1043

In [7]:
len(test_df['L-F_Pair'].unique())

261

In [9]:
 def create_prediction_columns(df, n):
        '''
        create the prediction pair by shifting the actual data up by the mentioned number(0.1*n seconds) to create the timeseries info
        '''
        df["nextframeAcc"] = df.groupby(
            ["L-F_Pair"], as_index=False)["v_Acc"].shift(-1*n)
        df["nextframesvel"] = df.groupby(
            ["L-F_Pair"], as_index=False)["v_Vel"].shift(-1*n)
        df["nextframeposition"] = df.groupby(
            ["L-F_Pair"], as_index=False)["Local_Y"].shift(-1*n)
        df["nextFrameSpacing"] = df.groupby(
            ["L-F_Pair"], as_index=False)["Space_Gap"].shift(-1*n)
        df["precnextframesvel"] = df.groupby(
            ["L-F_Pair"], as_index=False)["v_Vel_preceding"].shift(-1*n)
        df = df[df['nextframeposition'].notna()]
        df = df[df['nextframesvel'].notna()]
        df = df[df['nextframeAcc'].notna()]
        df = df[df['nextFrameSpacing'].notna()]
        df = df[df['precnextframesvel'].notna()]

        return df

In [10]:
train_df = create_prediction_columns(train_df,5)

In [9]:
test_df = create_prediction_columns(test_df,5)

In [10]:
train_df

,Vehicle_ID,Frame_ID,Local_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceding,...,pair_Time_Duration,total_pair_duration,Vehicle_Class,Space_Gap,Difference_of_speed,nextframeAcc,nextframesvel,nextframeposition,nextFrameSpacing,precnextframesvel
0,1760,5683,425.375767,6.49224,8.5,3,20.498565,-0.283574,1,1754,...,0.0,11.6,Heavy Vehicle,36.277296,1.090135,-0.679115,20.775275,415.056862,35.911536,22.732894
1,1760,5682,423.323483,6.49224,8.5,3,20.547120,-0.474961,1,1754,...,0.1,11.6,Heavy Vehicle,36.030408,1.327031,-0.626724,20.899921,412.973103,35.856672,22.853260
2,1760,5681,421.265679,6.49224,8.5,3,20.608958,-0.491087,1,1754,...,0.2,11.6,Heavy Vehicle,35.920680,1.527489,-0.649260,20.943266,410.880943,35.728656,22.929306
3,1760,5680,419.201729,6.49224,8.5,3,20.670048,-0.463286,1,1754,...,0.3,11.6,Heavy Vehicle,35.832288,1.700222,-0.481942,20.991363,408.784212,35.661600,22.960584
4,1760,5679,417.131926,6.49224,8.5,3,20.725999,-0.777079,1,1754,...,0.4,11.6,Heavy Vehicle,35.814000,1.844612,-0.714591,21.120779,404.573626,35.585400,22.892418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277227,1755,7317,432.553507,5.42544,8.4,2,13.974013,0.180951,1,1749,...,31.9,32.8,Car,16.831056,1.133864,0.372719,14.113733,439.571071,16.773144,15.278563
277228,1755,7318,433.951878,5.42544,8.4,2,13.993415,0.207085,1,1749,...,32.0,32.8,Car,16.821912,1.212407,0.369981,14.150868,440.984301,16.684752,15.222014
277229,1755,7319,435.352370,5.42544,8.4,2,14.016417,0.252962,1,1749,...,32.1,32.8,Car,16.843248,1.254982,0.370157,14.187875,442.401238,16.605504,15.138900
277230,1755,7320,436.755413,5.42544,8.4,2,14.044461,0.307920,1,1749,...,32.2,32.8,Car,16.858488,1.260305,0.568790,14.234822,443.822373,16.565880,15.030022


### SUPPORT VECTOR MACHINE

In [11]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

In [12]:
X_train = train_df.drop(['nextframeAcc', 'Vehicle_Class', 'Location', 'Period', 'L-F_Pair'], axis = 1)
y_train = train_df['nextframeAcc']
X_test = test_df.drop(['nextframeAcc', 'Vehicle_Class', 'Location', 'Period', 'L-F_Pair'], axis = 1)
y_test = test_df['nextframeAcc']

In [13]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [14]:
y_train

0        -0.679115
1        -0.626724
2        -0.649260
3        -0.481942
4        -0.714591
            ...   
277227    0.372719
277228    0.369981
277229    0.370157
277230    0.568790
277231    1.250826
Name: nextframeAcc, Length: 272064, dtype: float64

In [15]:
# Instantiate the Support Vector Regressor 
svr= SVR(C=1.0, epsilon=0.2)
 
# Fit the model
svr.fit(X_train_std, y_train)

SVR(epsilon=0.2)

In [16]:
# Make the predictions
y_predict = svr.predict(X_test_std)

### Calculating R2 and RMSE 

In [17]:
from sklearn.metrics import r2_score , mean_squared_error

mean_squared_error(y_test, y_predict)

0.17530829195378136

In [18]:
r2_score(y_test, y_predict)

0.5452114318304078

### Reaction time of 1s

In [11]:
train_df_1 = create_prediction_columns(train_df,10)

In [12]:
test_df_1 = create_prediction_columns(test_df,10)

In [13]:
X_train = train_df_1.drop(['nextframeAcc', 'Vehicle_Class', 'Location', 'Period', 'L-F_Pair'], axis = 1)
y_train = train_df_1['nextframeAcc']
X_test = test_df_1.drop(['nextframeAcc', 'Vehicle_Class', 'Location', 'Period', 'L-F_Pair'], axis = 1)
y_test = test_df_1['nextframeAcc']

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [19]:
# Instantiate the Support Vector Regressor 
svr= SVR(C=1.0, epsilon=0.2)
 
# Fit the model
svr.fit(X_train_std, y_train)

SVR(epsilon=0.2)

In [20]:
# Make the predictions
y_predict = svr.predict(X_test_std)

In [22]:
from sklearn.metrics import r2_score , mean_squared_error
r2_score(y_test, y_predict)

0.36624768493033644

In [23]:
mean_squared_error(y_test, y_predict)

0.23532245055355572

### Reaction Time 1.5 s

In [24]:
train_df_15 = create_prediction_columns(train_df,15)

In [25]:
test_df_15 = create_prediction_columns(test_df,15)

In [26]:
X_train = train_df_15.drop(['nextframeAcc', 'Vehicle_Class', 'Location', 'Period', 'L-F_Pair'], axis = 1)
y_train = train_df_15['nextframeAcc']
X_test = test_df_15.drop(['nextframeAcc', 'Vehicle_Class', 'Location', 'Period', 'L-F_Pair'], axis = 1)
y_test = test_df_15['nextframeAcc']

In [27]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [28]:
# Instantiate the Support Vector Regressor 
svr= SVR(C=1.0, epsilon=0.2)
 
# Fit the model
svr.fit(X_train_std, y_train)

SVR(epsilon=0.2)

In [29]:
# Make the predictions
y_predict = svr.predict(X_test_std)

In [30]:
r2_score(y_test, y_predict)

0.2951248208497299

In [31]:
mean_squared_error(y_test, y_predict)

0.2557275699067246

### Reaction Time of 2s

In [32]:
train_df_2= create_prediction_columns(train_df,20)

In [33]:
test_df_2= create_prediction_columns(test_df,20)

In [34]:
X_train = train_df_2.drop(['nextframeAcc', 'Vehicle_Class', 'Location', 'Period', 'L-F_Pair'], axis = 1)
y_train = train_df_2['nextframeAcc']
X_test = test_df_2.drop(['nextframeAcc', 'Vehicle_Class', 'Location', 'Period', 'L-F_Pair'], axis = 1)
y_test = test_df_2['nextframeAcc']

In [35]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [36]:
# Instantiate the Support Vector Regressor 
svr= SVR(C=1.0, epsilon=0.2)
 
# Fit the model
svr.fit(X_train_std, y_train)

SVR(epsilon=0.2)

In [37]:
# Make the predictions
y_predict = svr.predict(X_test_std)

In [38]:
r2_score(y_test, y_predict)

0.2607642183450698

In [39]:
mean_squared_error(y_test, y_predict)

0.2622074343817068